In [76]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# configurar salida de gráficos en el notebook
%matplotlib inline

In [ ]:
# cargar dataset desde archivo csv en dataframe 'df'
df = pd.read_csv('../data/insurance.csv')

# mostrar primeras 5 filas del dataframe
df.head()

In [ ]:
# mostrar información del dataframe (columnas, tipos, nulos)
df.info()

In [ ]:
# calcular matriz de correlación de columnas numéricas
df.corr(numeric_only=True)

In [ ]:
# configurar tamaño figura
plt.figure(figsize=(10, 6))
# gráfica mapa calor de correlaciones numéricas
sns.heatmap(df.corr(numeric_only=True).round(2), annot=True, cmap='viridis')
# mostrar figura
plt.show()

In [ ]:
# gráfica pairplot, observar distribución y relaciones
sns.pairplot(df)
plt.show()

In [ ]:
# gráfica pairplot, diferenciar variable 'smoker'
sns.pairplot(df, hue='smoker')
# mostrar figura
plt.show()

In [ ]:
# graficar relación entre 'age' y 'charges', diferenciar fumador
sns.relplot(data=df, x='age', y='charges', hue='smoker')
# mostrar figura
plt.show()

In [ ]:
# mostrar columna 'smoker'
df['smoker']

In [ ]:
# definir filtro para registros con 'smoker' == 'no'
filter = df['smoker'] == 'no'
# crear dataframe no fumadores
df_non_smokers = df[filter]
# mostrar primeras filas
df_non_smokers.head()

In [ ]:
# configurar tamaño figura
plt.figure(figsize=(10, 6))
# graficar mapa calor de correlaciones en no fumadores
sns.heatmap(df_non_smokers.corr(numeric_only=True).round(2), annot=True, cmap='viridis')
# mostrar figura
plt.show()

## Regresión simple

In [ ]:
# definir x como columna 'age' en no fumadores
X = df_non_smokers[['age']]
X.ndim

In [ ]:
# definir y como columna 'charges' en no fumadores
y = df_non_smokers['charges']
y.ndim

In [89]:
# crear modelo de regresión lineal
model = LinearRegression()

In [90]:
# importar función para separar dataset en entrenamiento y prueba
from sklearn.model_selection import train_test_split

# separar datos en entrenamiento (75%) y prueba (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# entrenar modelo con datos de entrenamiento
model.fit(X_train, y_train)

In [92]:
# predecir con datos de prueba
y_pred = model.predict(X_test)

In [ ]:
# calcular r2 y mae
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# imprimir resultados con 4 decimales
print('r2: ', round(r2, 4))
print('mae: ', round(mae, 4))

## Regresión múltiple

In [94]:
# definir x con 'age', 'bmi', 'children' en no fumadores
X = df_non_smokers[['age', 'bmi', 'children']] # input
# definir y como 'charges'
y = df_non_smokers['charges'] # output

In [95]:
# separar datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# crear modelo lineal y entrenar
model = LinearRegression()
model.fit(X_train, y_train)

# predecir valores
y_pred = model.predict(X_test)

# calcular r2 y mae
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# imprimir resultados
print('r2: ', round(r2, 4))
print('mae: ', round(mae, 4))

## Crear una nueva feature de entrada

In [ ]:
# mostrar columna 'smoker'
df['smoker']

In [ ]:
# transformar 'smoker' de categórico a numérico
df['smoker'].map({
    'yes': 1,
    'no': 0
})

In [ ]:
# asignar transformación a nueva columna 'smoker_int'
df['smoker_int'] = df['smoker'].map({
    'yes': 1,
    'no': 0
})
# mostrar primeras filas
df.head()

In [100]:
# definir x con 'age', 'bmi', 'children', 'smoker_int']]
X = df[['age', 'bmi', 'children', 'smoker_int']] # input
# definir y como 'charges'
y = df['charges'] # output

In [ ]:
# separar dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = LinearRegression()
# entrenar modelo
# model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# predecir con datos de prueba
y_pred = model.predict(X_test)

# calcular r2 y mae
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('r2: ', round(r2, 4))
print('mae: ', round(mae, 4))

In [ ]:
# obtener coeficientes del modelo
coefficients = model.coef_
coefficients

In [ ]:
# obtener intercept del modelo
intercept = model.intercept_
intercept

In [104]:
# definir función para calcular 'charges' usando coeficientes del modelo
def calculate_charges(age, bmi, children, smoker):
    return (age * coefficients[0]) + (bmi * coefficients[1]) + (children * coefficients[2]) + (smoker * coefficients[3]) + intercept

In [ ]:
# probar función con valores 19, 27.90, 0, 1
print(calculate_charges(19, 27.90, 0, 1))

In [ ]:
# comparar con predicción directa del modelo
model.predict([[19, 27.90, 0, 1]])